In [1]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='earthengine-theramed')

In [3]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [4]:
def calculate_ndvi_and_mask(image):
    ndvi = image.expression(
        "(nir - red) / (nir + red)",
        {
            'nir': image.select('B8'),
            'red': image.select('B4')
        }
    ).rename('NDVI')

    # Create a mask for NDVI values between -1 and 0
    # Filter out values 0.5 to 0.8
    ndvi_mask = ndvi.gte(0.5).And(ndvi.lte(0.8))

    # Apply the mask to the NDVI image
    ndvi_filtered = ndvi.updateMask(ndvi_mask)

    # ndvi_filtered = ndvi

    return ndvi_filtered



In [27]:
def getImage(province_name, start_date, end_date):
    fao_dataset = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")


    province = fao_dataset.filter(ee.Filter.eq('ADM1_NAME', province_name))

    # district_name = 'Buntharik'
    # district = fao_dataset.filter(ee.Filter.eq('ADM2_NAME', district_name))

    # Get the centroid of the province geometry.
    centroid = province.geometry().centroid()
    # centroid = district.geometry().centroid()

    # Define a buffer radius in meters (optional, if you want to create a ROI around the centroid).
    buffer_radius = 100000  # 100 km
    # buffer_radius = 10
    # buffer_radius = buffer_radius * 9
    # Create a buffer around the centroid to define the ROI.
    roi = centroid.buffer(buffer_radius)

    # Get the bounding box of the ROI
    bounding_box = roi.bounds()

    # Get the coordinates of the bounding box
    coordinates = bounding_box.coordinates().get(0).getInfo()

    # Extract the bounding box coordinates in the format [xmin, ymin, xmax, ymax]
    xmin = coordinates[0][0]
    ymin = coordinates[0][1]
    xmax = coordinates[2][0]
    ymax = coordinates[2][1]

    bounding_box_coordinates = [xmin, ymin, xmax, ymax]


    s2_image_collection = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
        .filterBounds(province)
        # .filterBounds(district)
        .sort("CLOUDY_PIXEL_PERCENTAGE")
        .map(mask_s2_clouds)
    )
    # Apply the NDVI calculation to the collection
    ndvi_collection = s2_image_collection.map(calculate_ndvi_and_mask)
    # Select a single image for demonstration purposes
    image = ndvi_collection.mean()
    # true_image = s2_image_collection.median().clip(province)
    # Clip the image to the ROI
    clipped_image = image.clip(province)
    # Define the custom NDVI color palette
    ndvi_palette = [
        '0000FF',  # Water
        'brown',   # Non-vegetated areas
        'yellow',  # Sparse vegetation
        'lightgreen',  # Moderate vegetation
        'darkgreen'   # Dense vegetation
    ]

    # Add the original NDVI layer to the map
    ndvi_params = {
        'bands': ['NDVI'],
        'min': -1,
        'max': 1,
        'palette': ndvi_palette
    }
    
    return clipped_image, ndvi_params, bounding_box_coordinates, roi



In [23]:
data_export = [
    {
        "province_name": "Udon Thani",
        "start_date": "2019-05-01",
        "end_date": "2019-06-01"
    }
]

In [29]:
i = data_export[0]
clipped_image, ndvi_params, bounding_box_coordinates, roi = getImage(i['province_name'], i['start_date'], i['end_date'])
# Display the inputs and the results.
m = geemap.Map()
m.addLayer(clipped_image, ndvi_params, 'NDVI')
m.centerObject(roi)
m
 

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [31]:
# Export the image
geemap.ee_to_geotiff(clipped_image, 'images/' + i['province_name'].lower().replace(" ", "-") + '-' + i['start_date'] + '-' + i['end_date'] + '.tif', bounding_box_coordinates, vis_params=ndvi_params, resolution=100)

Downloaded image 1/49
Downloaded image 2/49
Downloaded image 3/49
Downloaded image 4/49
Downloaded image 5/49
Downloaded image 6/49
Downloaded image 7/49
Downloaded image 8/49
Downloaded image 9/49
Downloaded image 10/49
Downloaded image 11/49
Downloaded image 12/49
Downloaded image 13/49
Downloaded image 14/49
Downloaded image 15/49
Downloaded image 16/49
Downloaded image 17/49
Downloaded image 18/49
Downloaded image 19/49
Downloaded image 20/49
Downloaded image 21/49
Downloaded image 22/49
Downloaded image 23/49
Downloaded image 24/49
Downloaded image 25/49
Downloaded image 26/49
Downloaded image 27/49
Downloaded image 28/49
Downloaded image 29/49
Downloaded image 30/49
Downloaded image 31/49
Downloaded image 32/49
Downloaded image 33/49
Downloaded image 34/49
Downloaded image 35/49
Downloaded image 36/49
Downloaded image 37/49
Downloaded image 38/49
Downloaded image 39/49
Downloaded image 40/49
Downloaded image 41/49
Downloaded image 42/49
Downloaded image 43/49
Downloaded image 44/